In [1]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing in

In [3]:
import kagglehub
from transformers import BlipForQuestionAnswering,AutoProcessor
from PIL import Image
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer,util
import re
import os
import transformers
from bert_score import score
from transformers import logging

logging.set_verbosity_error()
kagglehub.dataset_download("adityaav80/validatiioon")
kagglehub.dataset_download("hlgsagar1234567/vr-go")

def clean_text(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', str(text).lower().strip())

st,en = 0,1
start_row = int(50050/10)*st+1
end_row =  int(50050/10)*en
output_csv = f"/kaggle/working/blip_evaluation_{st}_{en}.csv"

baai = SentenceTransformer("BAAI/bge-base-en-v1.5")

processor = AutoProcessor.from_pretrained("Salesforce/blip-vqa-base", use_fast = True)
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")


df = pd.read_csv("/kaggle/input/validatiioon/val_red.csv")[:20]
results = []

image_path = None

for i in range(start_row, min(end_row, len(df))):
    try:
        if i % 10 == 0 or image_path is None:
            image_path = df.loc[i, "image_path"]
            image = Image.open(image_path).convert("RGB")
        
        question = df.loc[i, "question"]
        gt = df.loc[i, "answer"] 

        text = f"{question}"

        inputs = processor(images=image, text=text, return_tensors="pt")

        with torch.no_grad():
            outputs = model.generate(**inputs)
            prediction = processor.decode(outputs[0], skip_special_tokens=True)
            torch.cuda.empty_cache()

        pred_clean = clean_text(prediction)
        gt_clean = clean_text(gt)

        
        if pred_clean and gt_clean:
            emb_pred = baai.encode(pred_clean, convert_to_tensor=True, show_progress_bar=False)
            emb_gt = baai.encode(gt_clean, convert_to_tensor=True, show_progress_bar=False)
            sim = min(1.0, round(util.cos_sim(emb_pred, emb_gt).item(), 3))
            
            P, R, F1 = score([pred_clean], [gt_clean], lang='en', verbose=False)

            P_score = min(1.0, round(P.item(), 3))
            R_score = min(1.0, round(R.item(), 3))
            F1_score = min(1.0, round(F1.item(), 3))
            
        else:
            sim = ""

        exact_match = 1 if pred_clean == gt_clean and pred_clean else 0

        results.append({
            "index": i,
            "image_path": image_path,
            "question": question,
            "answer": gt,
            "blip_prediction": pred_clean,
            "baai_similarity": sim,
            "bertscore_precision_short": P_score,
            "bertscore_recall_short": R_score,
            "bertscore_f1_long": F1_score,
            "blip_exact_match_short": exact_match
        })
        
        if (i - start_row + 1) % 50 == 0:
                pd.DataFrame(results).to_csv(output_csv, mode='a', index=False, header=not os.path.exists(output_csv))
                print(f"[SAVED] Up to row {i}")
                results = []

    except Exception as e:
        print(f"[ERROR] at row {i}: {e}")
        continue

# Final save
if results:
    pd.DataFrame(results).to_csv(output_csv, mode='a', index=False, header=not os.path.exists(output_csv))
    print("Saved the file dude. Please Download!!!!!")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]